In [ ]:
%load_ext autoreload
%autoreload 2



    Let s = s0
    For k = 0 through kmax (exclusive):
        T ← temperature( 1 - (k+1)/kmax )
        Pick a random neighbour, snew ← neighbour(s)
        If P(E(s), E(snew), T) ≥ random(0, 1):
            s ← snew
    Output: the final state s



In [ ]:
from genetic_musical_generator.beat_fit import AnnealingFitter

class BassFit(AnnealingFitter):
    def __init__(self, track, anacrusis=4):
        super().__init__()
        self.track = track
        self.anacrusis = anacrusis
        
        self._lead_notes = None

In [ ]:
from genetic_musical_generator.beat_fit import BeatFit, play
from genetic_musical_generator.random_genome_to_midi import random_genome, genome2midi

In [ ]:
track = genome2midi('SDBDFDBBSFDSDDSFSDSUDSDBUBFUSBFFUDBDBUDU')
track

In [ ]:
beatfit = BeatFit(track)

In [ ]:
beatfit.fit()

In [ ]:
beatfit.fitted_state

In [ ]:
beatfit.tempo()

In [ ]:
beatfit.complete

In [ ]:
play(beatfit.complete)

Exclude notes from channel 9 from consideration.

In [ ]:
import random
from dataclasses import dataclass
from fastcore.basics import patch

In [ ]:
@patch(as_prop=True)
def total_beats(self:BassFit):
    return max([sum([m.time for m in t]) for t in self.track.tracks]) // self.track.ticks_per_beat

In [ ]:
@dataclass
class BassNote:
    pitch: int
    length: int

@patch
def neighbour(self:BassFit, state):
    """TODO: neighbouring states are a note extended or shortened at start or end, 
    or having a pitch changed"""
    if len(state)>0:
        i = random.randrange(len(state))
        note = state[i]
        options = [BassNote(note.pitch, note.length+m) for m in [-1,1] 
                   if note.length+m >= 2 and note.length+m <= 8]
        options.append(BassNote(random.randint(0,11), note.length))
        state[i] = random.choice(options)
    # check still as long as song length
    while sum([n.length for n in state]) < self.total_beats - self.anacrusis:
        state.append(BassNote(random.randint(0,11), random.randint(2,8)))
    return state

@patch
def random_state(self:BassFit):
    return self.neighbour([])

In [ ]:
bassfit = BassFit(beatfit.complete)
r = bassfit.random_state()
while r[0].length != 3:
    r = bassfit.random_state()
r

In [ ]:
n = bassfit.neighbour(r)
while len(n)<2:
    n = bassfit.neighbour(r)
n

In [ ]:
bassfit = BassFit(beatfit.complete)
r = bassfit.random_state()
while len(r)<2:
    r = bassfit.random_state()
r

In [ ]:
beatfit.complete

In [ ]:
from fractions import Fraction
from mido import MetaMessage

@dataclass
class Note:
    pitch: int
    start: Fraction
    end: Fraction

@patch(as_prop=True)
def lead_notes(self:BassFit):
    if self._lead_notes is not None: return self._lead_notes
    notes = []
    for track in self.track.tracks:
        pos = 0
        playing = dict()
        for msg in track:
            pos += msg.time
            if isinstance(msg, MetaMessage) or msg.channel == 9:
                continue
            if msg.type == 'note_on':
                if msg.note not in playing:
                    playing[msg.note] = pos
            elif msg.type == 'note_off':
                if msg.note not in playing:
                    continue
                notes.append(Note(msg.note, Fraction(playing[msg.note], self.track.ticks_per_beat), 
                                  Fraction(pos, self.track.ticks_per_beat)))
                del playing[msg.note]

        if len(playing)>0:
            raise ValueError('Not all notes were turned off')
            
    self._lead_notes = notes
    return notes

In [ ]:
beatfit.fitted_state = (Fraction(0, 1), Fraction(2, 1), 2)

In [ ]:
bassfit = BassFit(beatfit.complete)
bassfit.lead_notes

In [ ]:
class Cluster(set):
    def __init__(self, se=()):
        super().__init__([s%12 for s in se])
        
    def __le__(self, other):
        if super().issubset(other):
            return True
        for m in range(1,12):
            if set([(n+m)%12 for n in self]) <= other:
                return True
        return False

In [ ]:
assert Cluster([12,13,0,4,60]) == {0,1,4}

In [ ]:
assert Cluster([1,2]).issubset(Cluster([1,2,4]))
assert not Cluster([0,7]) <= Cluster([5,11])
assert Cluster([1,2]) <= Cluster([1,2,4])
assert Cluster([1,2]) <= Cluster([4,5,7])
assert Cluster([0,7]) <= Cluster([0,5])

In [ ]:
@patch
def coincident_notes(self:BassFit, start, end):
    return Cluster([n.pitch for n in 
                [n for n in self.lead_notes 
                 if (n.start >= start and n.start < end) or (n.start < start and n.end > start)]])

In [ ]:
assert bassfit.coincident_notes(5,6) == {6, 8}
assert bassfit.coincident_notes(3,4) == set()
assert bassfit.coincident_notes(4.5,5.5) == {6, 8, 9}

In [ ]:
assert Cluster([1,2,3]) | Cluster({12}) == {0,1,2,3}

In [ ]:
BassFit.dis_classes = [
    Cluster([0]),
    Cluster([0,7]),
    Cluster([0,4,7]),
    Cluster([0,3,7]),
    Cluster([0,4,7,9]),
    Cluster([0,2,4,7,9]),
    Cluster([0,2,4,7,9,11]),
    Cluster([0,2,4,6,7,9,11]),
    Cluster([0,2,4,6,8,9,11]),
    Cluster([0,2,3,6,7,9,11]),
    Cluster([0,2,4,6,8,10]),
    Cluster([0,2,3,5,6,8,9,11]),
    Cluster([0,2,4,6,7,8,9,11]),
    Cluster([0,2,3,4,6,7,8,9,11]),
    Cluster([0,2,3,4,6,7,8,9,10,11]),
    Cluster([0,2,3,4,5,6,7,8,9,10,11]),
    Cluster([0,1,2,3,4,5,6,7,8,9,10,11]),
]

@patch(cls_method=True)
def dissonance_class(cls:BassFit, cluster):
    for i,dc in enumerate(cls.dis_classes):
        if cluster <= dc:
            return i
        
    return len(cls.dis_classes)

In [ ]:
assert BassFit.dissonance_class(Cluster([54])) == 0
assert BassFit.dissonance_class(Cluster([54,59])) == 1
assert BassFit.dissonance_class(Cluster([18,19,20,21,22,23])) > BassFit.dissonance_class(Cluster([54,59]))

In [ ]:
@patch
def loss(self:BassFit, state):
    pos = self.anacrusis
    loss = 0
    for bass_note in state:
        cluster = self.coincident_notes(pos, pos+bass_note.length) | Cluster({bass_note.pitch})
        loss += BassFit.dissonance_class(cluster)
    return loss

@patch
def losses(self:BassFit, state, next_state):
    return self.loss(state), self.loss(next_state)

In [ ]:
bassfit = BassFit(beatfit.complete)
bassfit.fit()
bassfit.fitted_state

In [ ]:
random.gauss(0,12)

0 -> 36

...


4 -> 28

5 -> 29

...




In [ ]:
(lambda x: (x-4)%12 + 28)(4)

In [ ]:
list(range(28,53,12))

In [ ]:
import mido

@patch
def near_note(self:BassFit, note):
    # make a shot at desired pitch range with normal distribution skewed down a little
    shot = self.last_note + random.gauss(0,12) - 4
    
    # get note in bottom octave of bass
    bot_oct = lambda x: (x-4)%12 + 28
    
    # get the closest of note on bass neck to shot
    n = min(range(bot_oct(note), 53, 12), key=lambda x: abs(shot-x))
    self.last_note = n
    return n

In [ ]:
bassfit.last_note = 40

In [ ]:
bassfit.near_note(11)

In [ ]:
assert bassfit.near_note(0)%12 == 0
assert bassfit.near_note(11)%12 == 11

In [ ]:
import copy

@patch(as_prop=True)
def complete(self:BassFit):
    output = copy.deepcopy(self.track)
    bass = mido.MidiTrack()
    output.tracks.append(bass)
    
    # middle E on bass guitar
    self.last_note = 40
    
    pause = self.anacrusis * output.ticks_per_beat
    for note in self.fitted_state:
        pitch = self.near_note(note.pitch)
        bass.append(mido.Message('note_on', note=pitch, time=pause, channel=1))
        pause = 0
        bass.append(mido.Message('note_off', note=pitch, time=note.length*output.ticks_per_beat, channel=1))
        
    return output

In [ ]:
bassfit.complete

In [ ]:
play(bassfit.complete)